# Analysing Data

In [1]:
import pandas as pd

metad = pd.read_csv("data/HAM10000_metadata.csv")
metad

L8 = pd.read_csv("data/hmnist_8_8_L.csv")
RGB8 = pd.read_csv("data/hmnist_8_8_RGB.csv")
L28 = pd.read_csv("data/hmnist_28_28_L.csv")
RGB28 = pd.read_csv("data/hmnist_28_28_L.csv") 


metad.groupby("dx").count()
metad.groupby('lesion_id').count()
dupless = metad.drop_duplicates(subset=['lesion_id'])


metad= dupless.sort_values('lesion_id').reset_index(drop=True)

In [2]:
tags = metad.dx.unique().tolist()
tags

['nv', 'bkl', 'mel', 'akiec', 'bcc', 'df', 'vasc']

In [ ]:
metad

In [ ]:
_dict = metad.set_index('image_id').to_dict()['dx']
len(_dict)

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input


def preprocess(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
    img = preprocess_input(img)

    return img

In [ ]:
_dir = "data/HAM10000_images/"
ext = ".jpg"

all_images = []
categories = []

for key, value in _dict.items():
    #load image
    all_images.append(preprocess(_dir + key + ext))
    categories.append(value)

    #all_images.append(images)

all_images
tags

In [ ]:
for i in range(len(all_images)):
    print(f"{tags[i]} size = {len(all_images[i])}")

In [ ]:
categories

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

lbl = LabelEncoder()
int_enc = lbl.fit_transform(categories)

print(int_enc)


onehot_encoder = OneHotEncoder(sparse=False)
int_enc = int_enc.reshape(len(int_enc), 1)
onehot_encoded = onehot_encoder.fit_transform(int_enc)
print(onehot_encoded)


In [ ]:
bkl_ids = []
nv_ids = []
df_ids = []
mel_ids = []
vasc_ids = []
bcc_ids = []
akiec_ids = []

super_list = [bkl_ids,
              nv_ids,
              df_ids,
              mel_ids,
              vasc_ids,
              bcc_ids,
              akiec_ids]


for i in range(len(tags)):
    super_list[i] = (metad[metad['dx'] == tags[i]]['image_id'].values.tolist())

In [ ]:
super_list

In [ ]:
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

def get_preprocessed_images(images_ids: list, image_size: tuple) -> list:
    images = []
    dir1 = "data/HAM10000_images/"
    dir2 = "data/HAM10000_images_part_2/"
    ext = ".jpg"
    for _id in images_ids:
        try:
            img = image.load_img(dir1+_id+ext, target_size=image_size)
            img = image.img_to_array(img)
            img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
            img = preprocess_input(img)
            images.append(img)
        except:
            try:
                #continue
                img = image.load_img(dir2+_id+ext, target_size=image_size)
                img = image.img_to_array(img)
                img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
                img = preprocess_input(img)
                images.append(img)
            except:
                print(dir1 + _id)
                print(dir2 + _id)
                print("WOW")
            
            
    return np.vstack(images)

In [ ]:
# Import the keras preprocessing method.
from tensorflow.keras.preprocessing import image
image_size = (224, 224)
# Load your images and preprocess them.

bkl_images = get_preprocessed_images(super_list[0],image_size)
print("Done 0" )
nv_images = get_preprocessed_images(super_list[1],image_size)
print("Done 1")
df_images = get_preprocessed_images(super_list[2],image_size)
print("Done 2")
mel_images = get_preprocessed_images(super_list[3],image_size)
print("Done 3")
vasc_images = get_preprocessed_images(super_list[4],image_size)
print("Done 4")
bcc_images = get_preprocessed_images(super_list[5],image_size)
print("Done 5")
akiec_images = get_preprocessed_images(super_list[6],image_size)
print("Done 6")


In [ ]:
# Make a numpy array for each of the class labels (one hot encoded).
bkl_labels = np.tile([1, 0, 0, 0, 0, 0, 0], (bkl_images.shape[0], 1))
nv_labels = np.tile([0, 1, 0, 0, 0, 0, 0], (nv_images.shape[0], 1))
df_labels = np.tile([0, 0, 1, 0, 0, 0, 0], (df_images.shape[0], 1))
mel_labels = np.tile([0, 0, 0, 1, 0, 0, 0], (mel_images.shape[0], 1))
vasc_labels = np.tile([0, 0, 0, 0, 1, 0, 0], (vasc_images.shape[0], 1))
bcc_labels = np.tile([0, 0, 0, 0, 0, 1, 0], (bcc_images.shape[0], 1))
akiec_labels = np.tile([0, 0, 0, 0, 0, 0, 1], (akiec_images.shape[0], 1))

X = np.concatenate([bkl_images,
                    nv_images,
                    df_images,
                    mel_images,
                    vasc_images,
                    bcc_images,
                    akiec_images])

y = np.concatenate([bkl_labels,
                    nv_labels,
                    df_labels,
                    mel_labels,
                    vasc_labels,
                    bcc_labels,
                    akiec_labels])

print(X.shape, y.shape)


In [ ]:
#_X = np.vstack(all_images)

print(_X[0])
print(X[0])

print(y[0])
print(onehot_encoded[0])

In [ ]:
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, 
    y,
    test_size=0.2, 
    random_state=42, 
    shuffle=True
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, 
    y_train_val,
    test_size=0.2, 
    random_state=42, 
    shuffle=True
)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Determine the number of generated samples you want per original sample.
datagen_batch_size = 16

# Make a datagenerator object using ImageDataGenerator.
train_datagen = ImageDataGenerator(rotation_range=60,
                                    horizontal_flip=True)

# Feed the generator your train data.
train_generator = train_datagen.flow(X_train, y_train, batch_size=datagen_batch_size)

# Make a datagenerator object using ImageDataGenerator.
validation_datagen = ImageDataGenerator(rotation_range=60,
                                        horizontal_flip=True)

# Feed the generator your validation data.
validation_generator = validation_datagen.flow(X_val, y_val, batch_size=datagen_batch_size)

In [ ]:
# Import your chosen model!
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

# Make a model object. 
# Make sure you exclude the top part. set the input shape of the model to 224x224 pixels, with 3 color channels.
model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Freeze the imported layers so they cannot be retrained.
for layer in model.layers:
    layer.trainable = False
    
model.summary()

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout

new_model = Sequential()
new_model.add(model)
new_model.add(Flatten())
new_model.add(Dense(64, activation='relu'))
new_model.add(Dropout(0.5))
new_model.add(Dense(7, activation='sigmoid'))

# Summarize.
new_model.summary()

In [ ]:
# Compile and fit the model. Use the Adam optimizer and crossentropical loss. 
# Use the validation data argument during fitting to include your validation data.
new_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
history = new_model.fit(train_generator,
                        epochs=10, 
                        batch_size=500,
                        validation_data=validation_generator
                       )

In [ ]:
import tensorflow as tf
from matplotlib import pyplot as plt

def plot_history(history):
    """ This helper function takes the tensorflow.python.keras.callbacks.History
    that is output from your `fit` method to plot the loss and accuracy of
    the training and validation set.
    """
    fig, axs = plt.subplots(1,2, figsize=(12,6))
    axs[0].plot(history.history['accuracy'], label='training set')
    axs[0].plot(history.history['val_accuracy'], label = 'validation set')
    axs[0].set(xlabel = 'Epoch', ylabel='Accuracy', ylim=[0, .8])

    axs[1].plot(history.history['loss'], label='training set')
    axs[1].plot(history.history['val_loss'], label = 'validation set')
    axs[1].set(xlabel = 'Epoch', ylabel='Loss', ylim=[0, 11])
    
    axs[0].legend(loc='lower right')
    axs[1].legend(loc='upper right')
    
plot_history(history)

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_val)

print(y_val)
#cm = confusion_matrix(y_val,y_pred)

In [ ]:
#input random image
image_id = "ISIC_0029"
"""
for i in range(len(tags)):
        print(f"{tags[i]},",end="")
print("REAL")
"""
for i in range(306,500):
    real_id = image_id + str(i)
    
    filename = f"data/HAM10000_images/{real_id}.jpg"
    img = image.load_img(filename, target_size=image_size)
    print(f"{type(img)}")
    img = image.img_to_array(img)
    img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
    img = preprocess_input(img)

    pred = new_model.predict(img)
    pred = pred.tolist()[0]
    S = sum(pred)
    pred = [round(x / S,2) for x in pred]
    for i in range(len(tags)):
        print(f"{pred[i]},",end="")
    print(metad[metad['image_id'] == real_id]['dx'].values[0])